# Oscilloskope USBTCM utility

In [20]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob

# Step 1:
# Run "Record" on the oscilloscope
# and wait for 508 frames to be acquired.
# Step 2:
# Run this cell to initialize grabbing.
# This requires rw access to the /dev/usbtmc0 device.
# This can be granted by for example: 'sudo chmod a+rw /dev/usbtmc0'

class UsbTmcDriver:

    def __init__(self, device):
        self.device = device
        self.FILE = os.open(device, os.O_RDWR)
 
    def write(self, command):
        os.write(self.FILE, command);
 
    def read(self, length = 500):
        return os.read(self.FILE, length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope

# Looking for USBTMC device
def getDeviceList(): 
    dirList=os.listdir("/dev")
    result=list()

    for fname in dirList:
        if(fname.startswith("usbtmc")):
            result.append("/dev/" + fname)

    return result

# looking for oscilloscope
devices =  getDeviceList()
# initiate oscilloscope
osc = UsbTmcDriver(devices[0])


## Read repeatedly records from oscilloscope

This should be run after the initialization step.
Timeout at the end should be enlarged if not all 508 frames are transferred.

In [19]:
for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
    #print 'Deleting', f
    os.remove(f)

filename = 1
while True:
    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print 'XINC:', xinc,
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print 'YINC:', yinc,
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print 'TRIG:', trig,
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print 'YORIGIN:', yorig,
    

    osc.write(':FUNC:WREP:FEND?') # get number of last frame
    frames = int(osc.read(100))
    print 'FRAMES:', frames, 'SUBRUN', filename

    with h5py.File('./data/data'+str(filename)+'.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames)) # write number of frames
        hf.create_dataset('XINC', data=(xinc)) # write axis parameters
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        osc.write(':FUNC:WREP:FCUR 1') # skip to n-th frame
        time.sleep(0.5)
        for n in range(1,frames+1):
            osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
            time.sleep(0.1)

            osc.write(':WAV:DATA?') # read data
            #time.sleep(0.4)
            wave1 = bytearray(osc.read(500))
            wave2 = bytearray(osc.read(500))
            wave3 = bytearray(osc.read(500))
            #wave4 = bytearray(osc.read(500))
            #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
            wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
            hf.create_dataset(str(n), data=wave)
    filename = filename + 1
    osc.write(':FUNC:WREC:OPER REC') # start recording
    time.sleep(30) # delay for capturing


XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 1
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 2
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 3
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 4
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 5
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 6
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 7
XINC: 5e-08 YINC: 0.002 TRIG: 0.0162 YORIGIN: -79.0 FRAMES: 508 SUBRUN 8


KeyboardInterrupt: 